In [1]:
# import libraries
from bs4 import BeautifulSoup
#import numpy as np
import requests

In [2]:
url_base = 'https://codigo-postal.co/argentina/'
response = requests.get(url_base)
html = response.content
soup = BeautifulSoup(html, 'html.parser')

In [4]:
response.status_code

200

In [18]:
urlProvincias = []

In [3]:
# Encuentra los elementos que contienen los enlaces de las provincias
enlaces_provincias = soup.find_all('ul',{'class': 'column-list'})

In [19]:
for elemento in enlaces_provincias[0].findAll('a'):
    urlProvincias.append(elemento['href'])

In [21]:
len(urlProvincias)

24